## Import Library

In [62]:
# 제출 파일 생성 관련
import os
import zipfile

# 데이터 처리 및 분석
import pandas as pd
import numpy as np
from scipy import stats
from tqdm import tqdm

# 머신러닝 전처리
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder

# 머신러닝 모델
import xgboost as xgb

# 합성 데이터 생성
from sdv.metadata import SingleTableMetadata
from sdv.single_table import CTGANSynthesizer

# To ignore all warnings
import warnings
warnings.filterwarnings('ignore')

## 생성

### Load Data

In [11]:
# 경로 다를 수 있음 주의
train_all = pd.read_csv('./open/train.csv')
test_all = pd.read_csv('./open/test.csv')

train = train_all.drop(columns='ID')

train['Fraud_Type'].value_counts()

Fraud_Type
m    118800
a       100
j       100
h       100
k       100
c       100
g       100
i       100
b       100
f       100
d       100
e       100
l       100
Name: count, dtype: int64

### CTGAN을 사용한 데이터 생성

In [12]:
'''
(*) 리더보드 산식 중 생성데이터의 익명성(TCAP)채점을 위해 각 클래스 별로 1000개의 생성데이터가 반드시 필요합니다.
(*) 본 베이스 라인에서는 "Fraud_Type" 13종류에 대해 1000개씩 , 총 13,000개의 데이터를 생성할 예정입니다.
(*) 분류 모델 성능 개선을 위해 생성 데이터를 활용하는 것에는 생성 데이터의 Row 개수에 제한이 없습니다. 단, 리더보드 평가를 위해 제출을 하는 생성 데이터 프레임은 익명성(TCAP) 평가를 위함이며, 위의 조건을 갖춘 생성 데이터를 제출해야합니다.
'''
N_CLS_PER_GEN = 1000

# 이상치 처리 함수
def handle_outliers(series, n_std=3):
    mean = series.mean()
    std = series.std()
    z_scores = np.abs(stats.zscore(series))
    return series.mask(z_scores > n_std, mean)

# Time_difference 컬럼을 총 초로 변환 및 이상치 처리
train['Time_difference_seconds'] = pd.to_timedelta(train['Time_difference']).dt.total_seconds()
train['Time_difference_seconds'] = handle_outliers(train['Time_difference_seconds'])


# 모든 Fraud_Type 목록 생성 (m 포함)
fraud_types = train['Fraud_Type'].unique()

# 모든 합성 데이터를 저장할 DataFrame 초기화
all_synthetic_data = pd.DataFrame()

N_SAMPLE = 100

# 각 Fraud_Type에 대해 합성 데이터 생성 및 저장
for fraud_type in tqdm(fraud_types):
    
    # 해당 Fraud_Type에 대한 서브셋 생성
    subset = train[train["Fraud_Type"] == fraud_type]

    # 모든 Fraud_Type에 대해 100개씩 샘플링
    subset = subset.sample(n=N_SAMPLE, random_state=42)
    
    # Time_difference 열 제외 (초 단위로 변환된 컬럼만 사용)
    subset = subset.drop('Time_difference', axis=1)
    
    # 메타데이터 생성 및 모델 학습
    metadata = SingleTableMetadata()

    metadata.detect_from_dataframe(subset)
    metadata.set_primary_key(None)

    # 데이터 타입 설정
    column_sdtypes = {
        'Account_initial_balance': 'numerical',
        'Account_balance': 'numerical',
        'Customer_identification_number': 'categorical',  
        'Customer_personal_identifier': 'categorical',
        'Account_account_number': 'categorical',
        'IP_Address': 'ipv4_address',  
        'Location': 'categorical',
        'Recipient_Account_Number': 'categorical',
        'Fraud_Type': 'categorical',
        'Time_difference_seconds': 'numerical',
        'Customer_Birthyear': 'numerical'
    }

    # 각 컬럼에 대해 데이터 타입 설정
    for column, sdtype in column_sdtypes.items():
        metadata.update_column(
            column_name=column,
            sdtype=sdtype
        )
        
    synthesizer = CTGANSynthesizer(
                            metadata,
                            epochs=100
                        )
    synthesizer.fit(subset)

    synthetic_subset = synthesizer.sample(num_rows=N_CLS_PER_GEN)
    
    # 생성된 Time_difference_seconds의 이상치 처리
    synthetic_subset['Time_difference_seconds'] = handle_outliers(synthetic_subset['Time_difference_seconds'])
    
    # Time_difference_seconds를 다시 timedelta로 변환
    synthetic_subset['Time_difference'] = pd.to_timedelta(synthetic_subset['Time_difference_seconds'], unit='s')
    
    # Time_difference_seconds 컬럼 제거
    synthetic_subset = synthetic_subset.drop('Time_difference_seconds', axis=1)
    
    # 생성된 데이터를 all_synthetic_data에 추가
    all_synthetic_data = pd.concat([all_synthetic_data, synthetic_subset], ignore_index=True)
# 최종 결과 확인
print("\nFinal All Synthetic Data Shape:", all_synthetic_data.shape)
all_synthetic_data.head()

100%|███████████████████████████████████████████████████████████████████████████████| 13/13 [06:20<00:00, 29.26s/it]


Final All Synthetic Data Shape: (13000, 63)


,Customer_Birthyear,Customer_Gender,Customer_personal_identifier,Customer_identification_number,Customer_registration_datetime,Customer_credit_rating,Customer_flag_change_of_authentication_1,Customer_flag_change_of_authentication_2,Customer_flag_change_of_authentication_3,Customer_flag_change_of_authentication_4,...,Last_bank_branch_transaction_datetime,Flag_deposit_more_than_tenMillion,Unused_account_status,Recipient_account_suspend_status,Number_of_transaction_with_the_account,Transaction_history_with_the_account,First_time_iOS_by_vulnerable_user,Fraud_Type,Transaction_resumed_date,Time_difference
0,1973,female,김영호,dhITBu-DbhAPPn,2012-12-10 22:02:43,D,0,1,1,0,...,2022-01-06 01:18:56,0,1,1,0,0,0,m,2004-07-10 20:11:30,0 days 00:01:38
1,1951,male,이재현,VUdWiC-wXhKmwF,2006-09-26 20:25:20,A,1,1,0,1,...,2038-01-06 20:51:44,0,0,0,1,0,0,m,2004-07-10 20:11:30,0 days 00:01:38
2,1995,male,최유진,UWTuyh-pgXndzV,2009-05-06 23:11:50,A,1,1,1,1,...,2014-07-28 20:37:04,0,1,0,0,0,0,m,2010-01-19 20:41:03,0 days 00:01:38
3,1984,male,이민재,BDBAtF-ZmBUHYl,2003-07-13 16:06:24,B,1,1,0,1,...,2038-01-06 20:51:44,0,0,1,0,0,0,m,2008-11-18 10:26:32,0 days 00:01:38
4,1990,male,김영환,KpdklD-ymHOSLQ,2011-02-07 19:08:50,B,1,1,1,1,...,2036-10-16 16:07:25,0,1,1,0,3,0,m,2011-06-12 00:47:42,1 days 16:48:13


### 원본 데이터와 concat

In [13]:
origin_train = train_all.drop(columns="ID")
train_total = pd.concat([origin_train, all_synthetic_data])
train_total.shape

(133000, 63)

## 분류

### Data Preprocessing 1 : Select x, y

In [14]:
train_x = train_total.drop(columns=['Fraud_Type'])
train_y = train_total['Fraud_Type']

test_x = test_all.drop(columns=['ID'])

### Data Preprocessing 2 : 범주형 변수 인코딩

In [47]:
df_trainX = pd.DataFrame(train_x)
timedelta_columns = df_trainX.select_dtypes(include=['timedelta']).columns.tolist()
print('Timedelta columns:', timedelta_columns)

Timedelta columns: []


In [48]:
for column in df_trainX.columns:
    print(f'{column} types:')
    print(df_trainX[column].apply(type).value_counts())
    print()

Customer_Birthyear types:
Customer_Birthyear
<class 'int'>    133000
Name: count, dtype: int64

Customer_Gender types:
Customer_Gender
<class 'str'>    133000
Name: count, dtype: int64

Customer_personal_identifier types:
Customer_personal_identifier
<class 'str'>    133000
Name: count, dtype: int64

Customer_identification_number types:
Customer_identification_number
<class 'str'>    133000
Name: count, dtype: int64

Customer_registration_datetime types:
Customer_registration_datetime
<class 'str'>    133000
Name: count, dtype: int64

Customer_credit_rating types:
Customer_credit_rating
<class 'str'>    133000
Name: count, dtype: int64

Customer_flag_change_of_authentication_1 types:
Customer_flag_change_of_authentication_1
<class 'int'>    133000
Name: count, dtype: int64

Customer_flag_change_of_authentication_2 types:
Customer_flag_change_of_authentication_2
<class 'int'>    133000
Name: count, dtype: int64

Customer_flag_change_of_authentication_3 types:
Customer_flag_change_of_au

In [49]:
print(df_trainX['Time_difference'].apply(type).value_counts())

Time_difference
<class 'str'>                                         120000
<class 'pandas._libs.tslibs.timedeltas.Timedelta'>     13000
Name: count, dtype: int64


In [50]:
# Timedelta를 문자열로 변환
df_trainX['Time_difference'] = df_trainX['Time_difference'].astype(str)

In [51]:
print(df_trainX['Time_difference'].apply(type).value_counts())

Time_difference
<class 'str'>    133000
Name: count, dtype: int64


In [52]:
# 원본 보존하려고 새로운 변수 new_train_x로 선언
new_train_x = train_x.copy()
new_train_x['Time_difference'] = df_trainX['Time_difference'].tolist()

In [55]:
new_train_x['Time_difference']

0               0 days 02:53:50
1               0 days 01:07:33
2               0 days 00:52:59
3               0 days 01:24:05
4               0 days 01:43:29
                  ...          
12995           0 days 00:04:48
12996           0 days 00:04:48
12997           0 days 00:04:48
12998    0 days 05:21:59.739000
12999           0 days 00:04:48
Name: Time_difference, Length: 133000, dtype: object

In [57]:
le_subclass = LabelEncoder()
train_y_encoded = le_subclass.fit_transform(train_y)

# 변환된 레이블 확인
for i, label in enumerate(le_subclass.classes_):
    print(f"원래 레이블: {label}, 변환된 숫자: {i}")
    
# train_x
# 범주형 변수 인코딩
categorical_columns = train_x.select_dtypes(include=['object', 'category']).columns
ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

# 훈련 데이터 인코딩
train_x_encoded = train_x.copy()
train_x_encoded[categorical_columns] = ordinal_encoder.fit_transform(new_train_x[categorical_columns]) # 여기서 train_x를 new_train_x로 변경

# 특성 순서 저장
feature_order = train_x_encoded.columns.tolist()

원래 레이블: a, 변환된 숫자: 0
원래 레이블: b, 변환된 숫자: 1
원래 레이블: c, 변환된 숫자: 2
원래 레이블: d, 변환된 숫자: 3
원래 레이블: e, 변환된 숫자: 4
원래 레이블: f, 변환된 숫자: 5
원래 레이블: g, 변환된 숫자: 6
원래 레이블: h, 변환된 숫자: 7
원래 레이블: i, 변환된 숫자: 8
원래 레이블: j, 변환된 숫자: 9
원래 레이블: k, 변환된 숫자: 10
원래 레이블: l, 변환된 숫자: 11
원래 레이블: m, 변환된 숫자: 12


In [17]:
train_x_encoded

,Customer_Birthyear,Customer_Gender,Customer_personal_identifier,Customer_identification_number,Customer_registration_datetime,Customer_credit_rating,Customer_flag_change_of_authentication_1,Customer_flag_change_of_authentication_2,Customer_flag_change_of_authentication_3,Customer_flag_change_of_authentication_4,...,Unused_terminal_status,Last_atm_transaction_datetime,Last_bank_branch_transaction_datetime,Flag_deposit_more_than_tenMillion,Unused_account_status,Recipient_account_suspend_status,Number_of_transaction_with_the_account,Transaction_history_with_the_account,First_time_iOS_by_vulnerable_user,Transaction_resumed_date
0,1980,male,이상호,BJWQxd-WBASPLJ,2003-01-06 18:38:01,B,0,1,0,1,...,1,2003-01-22 23:38:48,2003-01-22 23:38:48,1,1,1,0,0,0,2003-01-22 23:38:48
1,1964,male,박상철,kurCwX-odPUXEt,2003-01-07 16:40:44,C,0,1,0,0,...,0,2003-01-21 21:29:08,2003-01-31 00:19:46,0,1,0,0,0,0,2003-01-19 21:29:08
2,1982,female,조옥자,OiERQa-CTXBoaX,2003-01-11 14:08:36,B,1,1,0,0,...,1,2003-01-31 07:13:28,2003-01-31 07:13:28,0,0,1,1,1,0,2003-01-31 07:13:28
3,1982,female,조옥자,OiERQa-CTXBoaX,2003-01-11 14:08:36,B,1,1,1,0,...,1,2003-01-31 11:49:56,2003-01-31 07:13:28,1,1,0,0,0,0,2003-01-31 07:13:28
4,1982,female,조옥자,OiERQa-CTXBoaX,2003-01-11 14:08:36,B,1,1,1,0,...,1,2003-01-31 11:49:56,2003-01-31 07:13:28,1,0,0,1,1,0,2003-01-31 07:13:28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12995,1960,female,서정웅,EedgrE-FgxNyrE,2009-05-25 05:39:44,B,0,1,1,1,...,1,2012-10-28 20:02:31,2030-01-16 00:43:01,1,1,1,1,1,0,2020-08-04 11:17:49
12996,1956,female,강영식,xPbHAe-DHGJjnf,2008-09-19 00:36:52,D,1,1,1,1,...,1,2019-09-09 16:06:32,2029-04-10 09:50:36,1,1,0,0,1,0,2022-02-17 13:38:40
12997,1950,male,박은주,koLbrU-TXAVGHt,2008-10-15 21:56:11,B,1,1,1,1,...,1,2012-10-28 20:02:31,2037-05-12 06:45:03,0,0,1,1,0,0,2022-11-23 15:12:24
12998,1959,male,황지후,tXjyoP-ZlorpbU,2011-03-10 19:19:48,B,0,1,1,1,...,1,2012-10-28 20:02:31,2036-08-15 23:53:02,1,0,0,0,2,0,2021-07-15 08:07:42


## Model

### Model Define

In [58]:
# 모델 정의 및 학습
model = xgb.XGBClassifier(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=6,
    random_state=42,
    use_label_encoder=False,
    eval_metric='mlogloss'
)
model.fit(train_x_encoded[feature_order], train_y_encoded)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.1, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=100,
              n_jobs=None, num_parallel_tree=None, objective='multi:softprob', ...)

## Inference

In [59]:
# 테스트 데이터 인코딩
test_x_encoded = test_x.copy()
test_x_encoded[categorical_columns] = ordinal_encoder.transform(test_x[categorical_columns])


# 특성 순서 맞추기 및 데이터 타입 일치
test_x_encoded = test_x_encoded[feature_order]
for col in feature_order:
    test_x_encoded[col] = test_x_encoded[col].astype(train_x_encoded[col].dtype)
    
# 예측
predictions = model.predict(test_x_encoded)
predictions_label = le_subclass.inverse_transform(predictions)

## Submission

In [61]:
# 분류 예측 결과 제출 데이터프레임(DataFrame)
# 분류 예측 결과 데이터프레임 파일명을 반드시 clf_submission.csv 로 지정해야합니다.
clf_submission = pd.read_csv("./sample_submission.csv")
clf_submission["Fraud_Type"] = predictions_label
clf_submission.head()
# 합성 데이터 생성 결과 제출 데이터프레임(DataFrame)
# 합성 데이터 생성 결과 데이터프레임 파일명을 반드시 syn_submission.csv 로 지정해야합니다.
all_synthetic_data.head()
'''
(*) 저장 시 각 파일명을 반드시 확인해주세요.
    1. 분류 예측 결과 데이터프레임 파일명 = clf_submission.csv
    2. 합성 데이터 생성 결과 데이터프레임 파일명 = syn_submission.csv

(*) 제출 파일(zip) 내에 두 개의 데이터프레임이 각각 위의 파일명으로 반드시 존재해야합니다.
(*) 파일명을 일치시키지 않으면 채점이 불가능합니다.
'''

# 폴더 생성 및 작업 디렉토리 변경
os.makedirs('./submission', exist_ok=True)
os.chdir("./submission/")

# CSV 파일로 저장
clf_submission.to_csv('./clf_submission.csv', encoding='UTF-8-sig', index=False)
all_synthetic_data.to_csv('./syn_submission.csv', encoding='UTF-8-sig', index=False)

# ZIP 파일 생성 및 CSV 파일 추가
with zipfile.ZipFile("../baseline_submission.zip", 'w') as submission:
    submission.write('clf_submission.csv')
    submission.write('syn_submission.csv')
    
print('Done.')

Done.
